In [7]:
"""
MTH-4150 Project 3

@author Will Mason Moses
@author Josh Lewis
@author Caitlin Chapman
@date 2021-10-28
"""
import numpy as np
from matplotlib import image, pyplot as plt
from math import floor, ceil


def backsub(U, b):
    """
    backsub(U,b)
    Solve the upper-triangular linear system with matrix U and right-hand side
    vector b.
    """
    n = len(b)
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        s = U[i, i + 1:] @ x[i + 1:]
        x[i] = (b[i] - s) / U[i, i]
    return x


def lsqrfact(A, b):
    """
    lsqrfact(A,b)
    Solve a linear least squares problem by QR factorization. Returns the
    minimizer of ||b-Ax||.
    """

    Q, R = np.linalg.qr(A, mode='reduced')
    c = Q.T @ b
    x = backsub(R, c)

    return x


def loadface(imagedir, subject, pose):
    """
    Load in the face for the given subject number (integer) and the given
    pose (integer). Directory of images is passed in as imagedir.
    """
    filename = f"{imagedir}/s{subject}/{pose}.pgm"
    image_vectors = image.imread(filename).astype(np.double)  # read, convert to double precision
    return image_vectors.flatten()


def showfaces(image_vectors):
    """
    Accepts a matrix of image vectors (assumed to be from 112 x 92 images, and
    with the image vectors as columns) and plots them in a grid.
    Will plot at most 16 faces.
    """
    n = image_vectors.shape[1]
    if n > 16:
        raise ValueError('A maximum of 16 faces please!')
    rows = ceil(n / 4)
    cols = 4

    fig = plt.figure(figsize=(10, 10))
    for j in range(n):
        pic = image_vectors[:, j].reshape(112, 92)
        ax = fig.add_subplot(rows, cols, j + 1)
        ax.axis('off')
        ax.imshow(pic, cmap='gray')

In [8]:
imagedir = 'attfaces-python'
all_faces = np.zeros([10304, 400])
all_face_id = np.zeros([400])
counter = 0
for subj in range(40):
    all_face_id[counter] = subj
    for pose in range(10):
        all_faces[:, counter] = loadface(imagedir, subj + 1, pose + 1)
        counter += 1

face_id = all_face_id[:150]
faces = np.zeros([10304, 150])
for subj in range(25):
    for pose in range(6):
        faces[:,(subj * 6) + pose] = all_faces[:,subj * 10 + pose]

Q_faces, R_faces = np.linalg.qr(faces, mode='reduced')


In [45]:
def identiface(Q, R, subj, z):
    """
    Calculates which face is most similar to the given image.
    :param Q: Q factor of face matrix
    :param R: R factor of face matrix
    :param subj: vector of subject numbers who's image appears in the corresponding column of the face matrix
    :param z: the image to find the similarity to
    :return: the most likely subject, and the confidence level in that similarity
    """
    qtz = Q.T @ z
    similarity = backsub(R, qtz)
    cum_sim = np.zeros([25])
    for subject in range(25):
        cum_sim[subject] = sum(similarity[(subject * 6):(subject * 6) + 6])
    # divides summed coefficients of chosen subject (max(cum_sim)) by 
    # the sum of all coefficients (abs value). If the coefficient of chosen
    # subject is 1, then confidence is 1/1. Abs value is taken so that all
    # contributions of other subjects reduce confidence. This effectively
    # accounts for how much the solution deviates from a full contribution by 
    # one subject
    confidence = max(cum_sim) / sum(abs(cum_sim))
    
    return np.argmax(cum_sim) + 1, confidence


identiface(Q_faces, R_faces, face_id, all_faces[:, 11:12])

(2, 0.999999999999976)

In [44]:
def performanceAnalyzer():
    accuracy = np.zeros([250,2])
    hits = 0
    misses = 0
    for face in range(250):
        accuracy[face] = identiface(Q_faces, R_faces, face_id, all_faces[:,face])
        # showfaces(all_faces[:,face * 10:face * 10 + 1])
    for result in range(accuracy.shape[0]):
        #print(accuracy[result,:])
        if accuracy[result,0] == -(-result // 10):
            hits += 1
        else:
            misses += 1
    print(hits, misses)

performanceAnalyzer()
# print(identiface(Q_faces, R_faces,face_id, all_faces[:, 5:6]))
# showfaces(all_faces[:, :10])

220 30
